In [6]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import os

def main():
    # === Load the dataset ===
    input_path = r"C:\Users\nakka\OneDrive\Documents\internship data.xlsx"
    print(f"📥 Loading data from: {input_path}")
    
    df = pd.read_excel(input_path)

    # ✅ Ensure amount is numeric
    df['amount'] = pd.to_numeric(df['amount'], errors='coerce')

    # === Normalize column names ===
    df.columns = [col.strip().lower() for col in df.columns]
    print(f"📌 Columns found: {list(df.columns)}")

    # === Clean data ===
    df = df.dropna(subset=['wallet', 'action', 'amount'])  # drop invalid rows
    df['action'] = df['action'].str.lower().str.strip()

    print("🔍 Extracting features from transactions...")

    # === Get list of unique wallets
    wallets = df['wallet'].unique()
    feature_rows = []

    for wallet in wallets:
        wallet_df = df[df['wallet'] == wallet]

        total_deposit = wallet_df[wallet_df['action'] == 'deposit']['amount'].sum()
        total_borrow = wallet_df[wallet_df['action'] == 'borrow']['amount'].sum()
        total_repay = wallet_df[wallet_df['action'] == 'repay']['amount'].sum()
        total_redeem = wallet_df[wallet_df['action'] == 'redeemunderlying']['amount'].sum()
        liquidation_count = (wallet_df['action'] == 'liquidationcall').sum()
        num_tx = len(wallet_df)

        borrow_deposit_ratio = total_borrow / (total_deposit + 1e-6)
        repay_borrow_ratio = total_repay / (total_borrow + 1e-6)

        feature_rows.append({
            'wallet': wallet,
            'total_deposit': total_deposit,
            'total_borrow': total_borrow,
            'total_repay': total_repay,
            'total_redeem': total_redeem,
            'liquidation_count': liquidation_count,
            'num_transactions': num_tx,
            'borrow_deposit_ratio': borrow_deposit_ratio,
            'repay_borrow_ratio': repay_borrow_ratio
        })

    features_df = pd.DataFrame(feature_rows)
    features_df = features_df.set_index('wallet')

    # === Scoring logic ===
    def calculate_score(row):
        score = 1000
        score -= 100 * row['liquidation_count']
        score -= 200 * max(0, 1 - row['repay_borrow_ratio'])
        score -= 100 * max(0, row['borrow_deposit_ratio'] - 1)
        return np.clip(score, 0, 1000)

    print("📊 Calculating credit scores...")
    features_df['raw_score'] = features_df.apply(calculate_score, axis=1)

    # Normalize scores to 0-1000 scale (optional)
    scaler = MinMaxScaler(feature_range=(0, 1000))
    features_df['credit_score'] = scaler.fit_transform(features_df[['raw_score']])

    # === Save the output ===
    output_path = r"C:\Users\nakka\OneDrive\Documents\internship\wallet_credit_scores.xlsx"
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    features_df[['credit_score']].to_excel(output_path)

    print(f"✅ Credit scores saved to: {output_path}")

if __name__ == "__main__":
    main()


📥 Loading data from: C:\Users\nakka\OneDrive\Documents\internship data.xlsx
📌 Columns found: ['wallet', 'timestamp', 'action', 'amount', 'create date', 'update date']
🔍 Extracting features from transactions...
📊 Calculating credit scores...
✅ Credit scores saved to: C:\Users\nakka\OneDrive\Documents\internship\wallet_credit_scores.xlsx
